In [1]:
import pandas as pd
import numpy as np
from scipy.stats import t

In [2]:
def family_of_loss_func(actual, predicted, degree):
    """
    Implemented from:
    Patton, A. J., 2011. Volatility forecasting comparison using imperfect 
    volatility proxies, Journal of Econometrics 160, 246-256.
    """
    if degree == -2:
        # QLIKE
        loss = actual / predicted - np.log(actual / predicted) - 1
    elif degree == -1:
        loss = predicted - actual + actual * np.log(actual / predicted)
    else:
        # MSE if degree = 0
        loss = (np.sqrt(actual) ** (2 * degree + 4) - predicted ** (degree + 2)) / ((degree + 1) * (degree + 2))
        loss -= (1 / (degree + 1)) * (predicted ** (degree + 1)) * (actual - predicted)
    return loss

def panel_DM(act, pred1, pred2, degree = 0):
    """
    Implemented from:
    Timmermann, A., Zhu, Y., 2019. Comparing Forecasting Performance with Panel Data
    """
    l1 = family_of_loss_func(act, pred1, degree)
    l2 = family_of_loss_func(act, pred2, degree)
    l1_mean = np.nanmean(l1)
    l2_mean = np.nanmean(l2)
    delta_l = l1 - l2
    n_T_sqrt = np.sqrt(np.sum(~np.isnan(delta_l)))
    sum_delta_l = np.nansum(delta_l)
    sigma_delta_l = np.nanstd(delta_l)
    j_dm = sum_delta_l / (sigma_delta_l * n_T_sqrt)
    p_value = 2 * t.cdf(-np.abs(j_dm), df = np.sum(~np.isnan(delta_l)) - 1)
    return j_dm, p_value


def panel_DM(act, pred1, pred2, degree = 0):
    l1 = family_of_loss_func(act, pred1, 0)
    l2 = family_of_loss_func(act, pred2, 0)
    delta_l = l1 - l2
    d_mean = np.nanmean(delta_l, axis = 0)
    n_T = np.sqrt(np.sum(~np.isnan(delta_l)))
    g0 = np.nanmean((delta_l - d_mean) * (delta_l - d_mean)) / n_T
    g1 = 2 * np.nanmean((delta_l[1:] - d_mean) * (delta_l[:-1] - d_mean)) / n_T
    DM = np.nanmean(delta_l) / np.sqrt((g0 + g1) / n_T)
    p_value = 2 * t.cdf(-np.abs(DM), df = (n_T ** 2) - 1)
    return DM, p_value

df_act = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/daily_vola.csv')
df_act.set_index(df_act.iloc[:, 0], inplace = True)
df_act = df_act.iloc[:-1, 1:]
df_act.pop('AMCR')
df_pred1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/pgarch_csa/forecasts_all.csv')
df_pred1.set_index(df_pred1.iloc[:, 0], inplace = True)
df_pred1 = df_pred1.iloc[:, 1:-1].T
df_pred2 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_EWMA/forecast.csv')
df_pred2.set_index(df_pred2.iloc[:, 0], inplace = True)
df_pred2 = df_pred2.iloc[:, 1:].T
df_pred2 = df_pred2.replace(1., np.nan)
df_pred3 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH/forecast_norm_dist.csv')
df_pred3.set_index(df_pred3.iloc[:, 0], inplace = True)
df_pred3 = df_pred3.iloc[:, 1:].T
df_pred3.pop('AMCR')
df_pred4 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH/forecast_t_dist.csv')
df_pred4.set_index(df_pred4.iloc[:, 0], inplace = True)
df_pred4 = df_pred4.iloc[:, 1:].T
df_pred4.pop('AMCR')
df_pred5 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH_MIDAS/Concated/forecast_norm.csv')
df_pred5.set_index(df_pred5.iloc[:, 0], inplace = True)
df_pred5 = df_pred5.iloc[:, 1:]
df_pred6 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/Panel_GARCH_MIDAS/Concated/forecast_stud.csv')
df_pred6.set_index(df_pred6.iloc[:, 0], inplace = True)
df_pred6 = df_pred6.iloc[:, 1:]

In [3]:
def panel_DM(act, pred1, pred2, degree = 0):
    l1 = family_of_loss_func(act, pred1, degree)
    l2 = family_of_loss_func(act, pred2, degree)
    delta = l1 - l2
    n_T = np.sqrt(np.sum(~np.isnan(delta)))
    DM = np.nansum(delta)/ n_T / np.sqrt(np.nansum(delta ** 2 / n_T))
    p_value = 2 * t.cdf(-np.abs(DM), df = (n_T ** 2) - 1)
    return DM, p_value

## Diebold-Mariano Test for Panel Forecast:

In [4]:
def panel_DM(act, pred1, pred2, degree = 0):
    l1 = family_of_loss_func(act, pred1, 0)
    l2 = family_of_loss_func(act, pred2, 0)
    delta_l = l1 - l2
    d_mean = np.nanmean(delta_l, axis = 0)
    n_T = np.sqrt(np.sum(~np.isnan(delta_l)))
    g0 = np.nanmean((delta_l - d_mean) * (delta_l - d_mean)) / n_T
    g1 = 2 * np.nanmean((delta_l[1:] - d_mean) * (delta_l[:-1] - d_mean)) / n_T
    DM = np.nanmean(delta_l) / np.sqrt((g0 + g1) / n_T)
    p_value = 2 * t.cdf(-np.abs(DM), df = (n_T ** 2) - 1)
    return DM, p_value

In [170]:
panel_DM(df_act.values, df_pred6.values, df_pred1.values)

(-5.372456573953989, 7.786691985418489e-08)

# Daily forecasts with the second edition of DM test:

### Panel GARCH with cross sectional adjustment vs EWMA:

In [4]:
panel_DM(df_act.values, df_pred1.values, df_pred2.values)

(0.4628941714825625, 0.6434413653854175)

### Panel GARCH (normal)  vs EWMA:

In [5]:
panel_DM(df_act.values, df_pred3.values, df_pred2.values)

(-0.09496718257654432, 0.9243411089159652)

### Panel GARCH (student-t) vs EWMA:

In [6]:
panel_DM(df_act.values, df_pred4.values, df_pred2.values)

(-0.13084761573373502, 0.895896152722059)

### Panel GARCH-MIDAS (normal) vs EWMA:

In [7]:
panel_DM(df_act.values, df_pred5.values, df_pred2.values)

(0.1840786111953204, 0.853952207925379)

### Panel GARCH-MIDAS (student-t) vs EWMA:

In [8]:
panel_DM(df_act.values, df_pred6.values, df_pred2.values)

(-0.019326673388581263, 0.9845805492475039)

### Panel GARCH (normal) vs GARCH with cross sectional adjustment

In [9]:
panel_DM(df_act.values, df_pred3.values, df_pred1.values)

(-0.3713712146771026, 0.7103619527524438)

### Panel GARCH (studnet-t) vs GARCH with cross sectional adjustment

In [10]:
panel_DM(df_act.values, df_pred4.values, df_pred1.values)

(-0.3815034611555416, 0.7028306206990484)

### Panel GARCH-MIDAS (normal) vs GARCH with cross sectional adjustment

In [11]:
panel_DM(df_act.values, df_pred5.values, df_pred1.values)

(-0.27862701628538816, 0.7805317294106787)

### Panel GARCH-MIDAS (student-t) vs GARCH with cross sectional adjustment

In [12]:
panel_DM(df_act.values, df_pred6.values, df_pred1.values)

(-0.3356519955293564, 0.7371341064566245)

# Results from the first version of DM test:

##### Note: Which is In [2]

In [5]:
print('Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_mse_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(7.551, 0.0)","(5.919, 0.0)","(6.12, 0.0)","(4.503, 0.0)","(5.372, 0.0)"
EWMA,,,"(1.635, 0.102)","(2.222, 0.026)","(-3.484, 0.0)","(0.341, 0.733)"
GARCH_NORM,,,,"(1.978, 0.048)","(-6.783, 0.0)","(-3.021, 0.003)"
GARCH_STUD,,,,,"(-7.02, 0.0)","(-3.676, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(8.479, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [6]:
print('Two-Daily MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_mse_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(10.179, 0.0)","(8.267, 0.0)","(8.501, 0.0)","(6.062, 0.0)","(7.331, 0.0)"
EWMA,,,"(1.278, 0.201)","(2.329, 0.02)","(-5.01, 0.0)","(-0.523, 0.601)"
GARCH_NORM,,,,"(3.543, 0.0)","(-7.375, 0.0)","(-3.07, 0.002)"
GARCH_STUD,,,,,"(-7.765, 0.0)","(-4.242, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(10.158, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [7]:
print('Weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_mse_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(8.516, 0.0)","(7.154, 0.0)","(7.369, 0.0)","(7.972, 0.0)","(7.778, 0.0)"
EWMA,,,"(-0.79, 0.43)","(1.752, 0.08)","(-5.504, 0.0)","(0.498, 0.618)"
GARCH_NORM,,,,"(10.318, 0.0)","(-3.382, 0.001)","(1.618, 0.106)"
GARCH_STUD,,,,,"(-4.423, 0.0)","(-1.767, 0.077)"
GARCH_MIDAS_NORM,,,,,,"(5.919, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [8]:
print('Two-weekly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_mse_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-weekly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(12.015, 0.0)","(10.205, 0.0)","(10.647, 0.0)","(11.792, 0.0)","(11.52, 0.0)"
EWMA,,,"(-2.685, 0.007)","(1.579, 0.114)","(-3.616, 0.0)","(2.266, 0.023)"
GARCH_NORM,,,,"(15.013, 0.0)","(-1.623, 0.105)","(6.664, 0.0)"
GARCH_STUD,,,,,"(-4.103, 0.0)","(1.282, 0.2)"
GARCH_MIDAS_NORM,,,,,,"(8.205, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [9]:
print('Monthly MSE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/monthly_mse_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly MSE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(13.133, 0.0)","(11.328, 0.0)","(11.806, 0.0)","(13.635, 0.0)","(13.082, 0.0)"
EWMA,,,"(-4.236, 0.0)","(0.55, 0.583)","(-2.639, 0.008)","(2.043, 0.041)"
GARCH_NORM,,,,"(14.065, 0.0)","(0.17, 0.865)","(9.334, 0.0)"
GARCH_STUD,,,,,"(-3.011, 0.003)","(2.472, 0.013)"
GARCH_MIDAS_NORM,,,,,,"(8.179, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [10]:
print('Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/daily_qlike_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(7.551, 0.0)","(5.919, 0.0)","(6.12, 0.0)","(4.503, 0.0)","(5.372, 0.0)"
EWMA,,,"(1.635, 0.102)","(2.222, 0.026)","(-3.484, 0.0)","(0.341, 0.733)"
GARCH_NORM,,,,"(1.978, 0.048)","(-6.783, 0.0)","(-3.021, 0.003)"
GARCH_STUD,,,,,"(-7.02, 0.0)","(-3.676, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(8.479, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [12]:
print('Two-Daily QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_daily_qlike_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Daily QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(10.179, 0.0)","(8.267, 0.0)","(8.501, 0.0)","(6.062, 0.0)","(7.331, 0.0)"
EWMA,,,"(1.278, 0.201)","(2.329, 0.02)","(-5.01, 0.0)","(-0.523, 0.601)"
GARCH_NORM,,,,"(3.543, 0.0)","(-7.375, 0.0)","(-3.07, 0.002)"
GARCH_STUD,,,,,"(-7.765, 0.0)","(-4.242, 0.0)"
GARCH_MIDAS_NORM,,,,,,"(10.158, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [13]:
print('Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/weekly_qlike_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(8.516, 0.0)","(7.154, 0.0)","(7.369, 0.0)","(7.972, 0.0)","(7.778, 0.0)"
EWMA,,,"(-0.79, 0.43)","(1.752, 0.08)","(-5.504, 0.0)","(0.498, 0.618)"
GARCH_NORM,,,,"(10.318, 0.0)","(-3.382, 0.001)","(1.618, 0.106)"
GARCH_STUD,,,,,"(-4.423, 0.0)","(-1.767, 0.077)"
GARCH_MIDAS_NORM,,,,,,"(5.919, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [14]:
print('Two-Weekly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/two_weekly_qlike_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Two-Weekly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(12.015, 0.0)","(10.205, 0.0)","(10.647, 0.0)","(11.792, 0.0)","(11.52, 0.0)"
EWMA,,,"(-2.685, 0.007)","(1.579, 0.114)","(-3.616, 0.0)","(2.266, 0.023)"
GARCH_NORM,,,,"(15.013, 0.0)","(-1.623, 0.105)","(6.664, 0.0)"
GARCH_STUD,,,,,"(-4.103, 0.0)","(1.282, 0.2)"
GARCH_MIDAS_NORM,,,,,,"(8.205, 0.0)"
GARCH_MIDAS_STUD,,,,,,


In [15]:
print('Monthly QLIKE:')
print('Note: (DM test value, p value)')
mse1 = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/panel_dm/Monthly_qlike_v2.csv')
mse1.set_index(mse1.iloc[:, 0], inplace = True)
mse1 = mse1.iloc[:, 1:].fillna('')
mse1

Monthly QLIKE:
Note: (DM test value, p value)


,GARCH_CSA,EWMA,GARCH_NORM,GARCH_STUD,GARCH_MIDAS_NORM,GARCH_MIDAS_STUD
Unnamed: 0,,,,,,
GARCH_CSA,,"(13.133, 0.0)","(11.328, 0.0)","(11.806, 0.0)","(13.635, 0.0)","(13.082, 0.0)"
EWMA,,,"(-4.236, 0.0)","(0.55, 0.583)","(-2.639, 0.008)","(2.043, 0.041)"
GARCH_NORM,,,,"(14.065, 0.0)","(0.17, 0.865)","(9.334, 0.0)"
GARCH_STUD,,,,,"(-3.011, 0.003)","(2.472, 0.013)"
GARCH_MIDAS_NORM,,,,,,"(8.179, 0.0)"
GARCH_MIDAS_STUD,,,,,,
